In [1]:
import pandas as pd


In [2]:
def read_data_in_chunk():
    df_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize=5000)
    return df_iter


print mssing value counts, memory footprint, total memory

In [3]:
df_iter = read_data_in_chunk()

total_mem = 0
null_counts = {}

for df in df_iter:
    for c in df.columns:
        if c not in null_counts:
            null_counts[c] = df[c].isnull().sum()
        else:
            null_counts[c] += df[c].isnull().sum()
        
    mem = df.memory_usage(deep=True).sum()/(1024*1024)
    total_mem += mem
    print("Mem (MB): ", mem)
    
    
print("Total Mem (MB): ", total_mem)

print("Null count for each column:")
for c in sorted(null_counts):
    print(null_counts[c], "\t <== ", c)
    

Mem (MB):  5.57919502258
Mem (MB):  5.5281867981
Mem (MB):  5.53500461578
Mem (MB):  5.52816295624
Mem (MB):  5.52430725098
Mem (MB):  5.55341243744
Mem (MB):  5.5313911438
Mem (MB):  5.50961303711
Mem (MB):  5.39609050751
Mem (MB):  4.63945198059
Mem (MB):  2.66366863251
Total Mem (MB):  56.9884843826
Null count for each column:
643 	 <==  company_category_code
533 	 <==  company_city
1 	 <==  company_country_code
1 	 <==  company_name
1 	 <==  company_permalink
1 	 <==  company_region
492 	 <==  company_state_code
3 	 <==  funded_at
3 	 <==  funded_month
3 	 <==  funded_quarter
3 	 <==  funded_year
3 	 <==  funding_round_type
50427 	 <==  investor_category_code
12480 	 <==  investor_city
12001 	 <==  investor_country_code
2 	 <==  investor_name
2 	 <==  investor_permalink
2 	 <==  investor_region
16809 	 <==  investor_state_code
3599 	 <==  raised_amount_usd


print default datatype\columns

In [4]:
df_iter = read_data_in_chunk()


for df in df_iter:
    print("dtypes: ", df.dtypes)
    print("columns: ", df.columns.tolist())
    break

dtypes:  company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object
columns:  ['company_permalink', 'company_name', 'company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_permalink', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_ci

tight numerical columm representation types

In [5]:
df_iter = read_data_in_chunk()

total_mem = 0

for i, df in enumerate(df_iter):
    
    df.funded_year = pd.to_numeric(df.funded_year, downcast='integer')
    df.raised_amount_usd = pd.to_numeric(df.raised_amount_usd, downcast='float')
 
    print(df.funded_year.dtype)
    print(df.raised_amount_usd.dtype)
    
    mem = df.memory_usage(deep=True).sum()/(1024*1024)
    total_mem += mem
    
    print("Mem (MB): ", mem)
    
    
print("Total Mem (MB): ", total_mem)

int16
float32
Mem (MB):  5.53151130676
int16
float32
Mem (MB):  5.48050308228
int16
float32
Mem (MB):  5.48732089996
int16
float32
Mem (MB):  5.48047924042
int16
float32
Mem (MB):  5.47662353516
int16
float32
Mem (MB):  5.50572872162
float64
float32
Mem (MB):  5.51231765747
int16
float32
Mem (MB):  5.46192932129
int16
float32
Mem (MB):  5.34840679169
int16
float32
Mem (MB):  4.59176826477
int16
float32
Mem (MB):  2.63629817963
Total Mem (MB):  56.512887001


## df.funded_year ==> int16
## df.raised_amount_usd ==> float32
is ok for all chunks

In [6]:
df_iter = read_data_in_chunk()

unique_vals = {}
all_columns = []

for i, df in enumerate(df_iter):
    
    obj_cols = df.select_dtypes(include=['object'])
    if i == 0:
        print(obj_cols.head(5))
    
    for c in obj_cols.columns:
        if c not in unique_vals:
            unique_vals[c] = []
            all_columns.append(c)
            
        unique_vals[c].append(obj_cols[c].value_counts())
    
print(all_columns)

print("\n\n\n\nEvaluate unique value less than 50% of total data counts\n")
to_cat_cols = []

for c in unique_vals:
    unique_vals[c] = pd.concat(unique_vals[c])
    unique_vals[c] = unique_vals[c].groupby(unique_vals[c].index).sum()
    
    if len(unique_vals[c])/unique_vals[c].sum() < 0.5:
        to_cat_cols.append(c)
        
print("Columns that should be convert to category type:\n", to_cat_cols)        

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

In [7]:
useful_cols = ['company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_city', 'funding_round_type', 'funded_at','raised_amount_usd']
to_dat_cols = ['funded_at']
to_cat_cols = ['investor_category_code', 'funding_round_type', 'investor_region', 'investor_name', 'company_category_code', 'investor_state_code', 'company_country_code', 'company_region', 'investor_city', 'company_city', 'investor_country_code', 'company_state_code']

In [8]:
to_cat_cols_dict = {}

for c in to_cat_cols:
    to_cat_cols_dict[c] = 'category'

to_cat_cols_dict["raised_amount_usd"] = 'float32'


In [9]:
def refined_read_data_in_chunk():
    df_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize=5000, usecols=useful_cols, parse_dates=to_dat_cols, dtype=to_cat_cols_dict)
    return df_iter

In [10]:
df_iter = refined_read_data_in_chunk()

total_mem = 0

for i, df in enumerate(df_iter):
    
    mem = df.memory_usage(deep=True).sum()/(1024*1024)
    total_mem += mem
    print("Mem (MB): ", mem)
    
    
print("Total Mem (MB): ", total_mem)

df_iter = refined_read_data_in_chunk()
for df in df_iter:
    print("dtypes: ", df.dtypes)
    print("columns: ", df.columns.tolist())
    break

Mem (MB):  0.32080745697
Mem (MB):  0.248866081238
Mem (MB):  0.245860099792
Mem (MB):  0.226246833801
Mem (MB):  0.239699363708
Mem (MB):  0.237704277039
Mem (MB):  0.249403953552
Mem (MB):  0.240126609802
Mem (MB):  0.260649681091
Mem (MB):  0.315644264221
Mem (MB):  0.186584472656
Total Mem (MB):  2.77159309387
dtypes:  company_category_code           category
company_country_code            category
company_state_code              category
company_region                  category
company_city                    category
investor_name                   category
investor_category_code          category
investor_country_code           category
investor_state_code             category
investor_city                   category
funding_round_type              category
funded_at                 datetime64[ns]
raised_amount_usd                float32
dtype: object
columns:  ['company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_na

In [11]:
import sqlite3
conn = sqlite3.connect('crunch.db')

df_iter = refined_read_data_in_chunk()

for df in df_iter:
    df.to_sql("crunch_table", conn, if_exists='append', index=False)
    
results_df = pd.read_sql('PRAGMA table_info(crunch_table);', conn)
print(results_df)  


    cid                    name       type  notnull dflt_value  pk
0     0   company_category_code       TEXT        0       None   0
1     1    company_country_code       TEXT        0       None   0
2     2      company_state_code       TEXT        0       None   0
3     3          company_region       TEXT        0       None   0
4     4            company_city       TEXT        0       None   0
5     5           investor_name       TEXT        0       None   0
6     6  investor_category_code       TEXT        0       None   0
7     7   investor_country_code       TEXT        0       None   0
8     8     investor_state_code       TEXT        0       None   0
9     9           investor_city       TEXT        0       None   0
10   10      funding_round_type       TEXT        0       None   0
11   11               funded_at  TIMESTAMP        0       None   0
12   12       raised_amount_usd       REAL        0       None   0


In [20]:
!wc --help
!wc -c crunch.db

Usage: wc [OPTION]... [FILE]...
  or:  wc [OPTION]... --files0-from=F
Print newline, word, and byte counts for each FILE, and a total line if
more than one FILE is specified.  With no FILE, or when FILE is -,
read standard input.  A word is a non-zero-length sequence of characters
delimited by white space.
The options below may be used to select which counts are printed, always in
the following order: newline, word, character, byte, maximum line length.
  -c, --bytes            print the byte counts
  -m, --chars            print the character counts
  -l, --lines            print the newline counts
      --files0-from=F    read input from the files specified by
                           NUL-terminated names in file F;
                           If F is - then read names from standard input
  -L, --max-line-length  print the length of the longest line
  -w, --words            print the word counts
      --help     display this help and exit
      --version  output version information 

In [21]:
print("Total MB for crunch.db:", 24479744/1024**2)


Total MB for crunch.db: 23.345703125


In [24]:
crunch_df = pd.read_sql('select company_category_code, investor_name, raised_amount_usd, funding_round_type from crunch_table;', conn)
print(crunch_df.head())  

  company_category_code      investor_name  raised_amount_usd  \
0           advertising  1-800-FLOWERS.COM          2000000.0   
1                  news        10Xelerator            20000.0   
2             messaging        10Xelerator            20000.0   
3              software        10Xelerator            20000.0   
4                   web        10Xelerator            20000.0   

  funding_round_type  
0           series-a  
1              other  
2              other  
3              angel  
4              other  


In [31]:
cmp_type_df = crunch_df.groupby(crunch_df.company_category_code).sum()
cmp_type_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 

print(cmp_type_df.head())

                       raised_amount_usd
company_category_code                   
biotech                     4.415857e+11
software                    2.923381e+11
mobile                      2.591095e+11
cleantech                   2.108209e+11
enterprise                  1.834437e+11


Which category of company attracted the most investments? ==> Biotech

In [32]:
investor_name_df = crunch_df.groupby(crunch_df.investor_name).sum()
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 

print(investor_name_df.head())

                                  raised_amount_usd
investor_name                                      
Kleiner Perkins Caufield & Byers       4.487131e+10
New Enterprise Associates              3.877017e+10
Accel Partners                         2.588850e+10
Goldman Sachs                          2.550184e+10
Sequoia Capital                        2.415761e+10


Which investor contributed the most money (across all startups)? ==> Kleiner Perkins Caufield & Byers

In [46]:
investor_name_counts = crunch_df.investor_name.value_counts()

investor_name_df["count"] = investor_name_counts
investor_name_df["avg investment"] = investor_name_df['raised_amount_usd'] / investor_name_df["count"] 
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 
print(investor_name_df.head())

                                  raised_amount_usd  avg investment  count
investor_name                                                             
Kleiner Perkins Caufield & Byers       4.487131e+10    2.854409e+07   1572
New Enterprise Associates              3.877017e+10    2.178099e+07   1780
Accel Partners                         2.588850e+10    2.009977e+07   1288
Goldman Sachs                          2.550184e+10    5.183300e+07    492
Sequoia Capital                        2.415761e+10    1.636694e+07   1476


Which investors contributed the most money per startup? ==> still Kleiner Perkins Caufield & Byers

In [58]:
funding_round_type_counts = crunch_df.funding_round_type.value_counts()
funding_round_type_df = pd.DataFrame()
funding_round_type_df['count'] = funding_round_type_counts
funding_round_type_df.sort_values('count', ascending=False, inplace=True) 

print(funding_round_type_df.head())

print(funding_round_type_df.tail())

           count
series-a   55752
series-c+  43480
angel      35956
venture    35668
series-b   35176
                count
series-b        35176
other            3856
private-equity   1428
post-ipo          132
crowdfunding       20


Which funding round was the most popular? ==> series-a
Which was the least popular? ==> crowdfunding       